# Preprocesamiento

En este notebook se preprocesan todos los datos obtenidos en el paso uno y se preparan para poder entrar los modelos.

In [1]:
import pandas as pd

In [6]:
df_precios_md = pd.read_csv('../datos/precios_md_OMIE.csv')
df_precios_gas = pd.read_csv('../datos/precios_gas.csv')
df_demanda_prev = pd.read_csv('../datos/demanda_prevision_D-1_ESIOS.csv')
df_demanda_real = pd.read_csv('../datos/demanda_real_ESIOS.csv')
df_temp_radi = pd.read_csv('../datos/temperaturas_radiacion.csv')
df_precios_fr = pd.read_csv('../datos/precios_md_Francia.csv')
df_tipo_dias = pd.read_csv('../datos/tipo_dias.csv')

dfs_diarios = [df_precios_gas, df_temp_radi, df_tipo_dias]
dfs_horarios = [df_precios_md, df_demanda_prev, df_demanda_real, df_precios_fr]

### Expandir los datos diarios a horarios

In [7]:
# Expandir datos diarios a horarios
for df in dfs_diarios:
    df_horario = df.loc[df.index.repeat(24)].reset_index(drop=True)
    df_horario['hora'] = df_horario.groupby('fecha').cumcount()
    df_horario['fecha'] = pd.to_datetime(df_horario['fecha']) + pd.to_timedelta(df_horario['hora'], unit='h')
    df_horario.drop(columns=['hora'], inplace=True)
    dfs_horarios.append(df_horario)

In [8]:
# Establecer el tipo fecha
for df in dfs_horarios:
    df['fecha'] = pd.to_datetime(df['fecha'])


## Unir todos los dfs

In [9]:
from functools import reduce

df = reduce(lambda left, right: pd.merge(left, right, on='fecha', how='inner'), dfs_horarios)

df

,fecha,precio_MD,demanda_prevision_D-1,demanda_Real,precio_Francia,precio_GAS,Temperatura,Radiacion,tipo_dia
0,2020-01-01 00:00:00,41.88,22733.0,18594.922,38.60,11.89,5.65,2.5531,F
1,2020-01-01 01:00:00,38.60,21593.0,17466.294,36.55,11.89,5.65,2.5531,F
2,2020-01-01 02:00:00,36.55,20362.0,16426.445,32.32,11.89,5.65,2.5531,F
3,2020-01-01 03:00:00,32.32,19604.0,15775.322,30.85,11.89,5.65,2.5531,F
4,2020-01-01 04:00:00,30.85,19386.0,15580.377,30.14,11.89,5.65,2.5531,F
...,...,...,...,...,...,...,...,...,...
43795,2024-12-30 19:00:00,172.72,32302.0,28197.217,139.39,48.60,2.84,2.6006,L
43796,2024-12-30 20:00:00,167.21,31700.5,27740.899,130.74,48.60,2.84,2.6006,L
43797,2024-12-30 21:00:00,155.65,29118.8,25678.627,128.95,48.60,2.84,2.6006,L
43798,2024-12-30 22:00:00,146.13,26335.8,23061.434,126.45,48.60,2.84,2.6006,L


In [10]:
df.to_csv("../datos/dataset.csv", index=False)